In [70]:
import pandas as pd
from scipy.stats import pearsonr
import numpy as np
import re

In [71]:
raw_df = pd.read_csv("results/survey1_pilot_v1.csv")

In [72]:
short_col_df = raw_df.rename(lambda s: s[0:3], axis='columns')
short_col_df

,Tri,Sce,001,002,003,004,005,006,007,008,...,019,020,021,022,023,024,025,026,027,028
0,NaN,NaN,01 Distributive Price Fairness Perception,01 Distributive Price Fairness Perception,01 Distributive Price Fairness Perception,03 Purchase Intention,03 Purchase Intention,03 Purchase Intention,04 Future Search Intention,04 Future Search Intention,...,13 Money Spent for Online Purchases,14 Social Media Usage,15 Manipulation Check Price Direction,17 Realism Check,17 Realism Check,18 Demographics,18 Demographics,18 Demographics,18 Demographics,18 Demographics
1,0.0,Disadvantaged Customer,4,5,4,5,6,5,5,5,...,d)101–200$,a)Facebook b)Instagram c)Twitter,b) $100,2,4,a,24,a)Some school but no degree,a)Employed full-time,"e)$75,000 to $99,999"
2,1.0,Disadvantaged Customer,2,4,5,4,3,5,5,6,...,c)51–100$,a)\nb),b) $100,5,5,a)Male,25,d)Bachelor’s degree,a)Employed full-time,"d) $50,000 to $74,999"
3,2.0,Disadvantaged Customer,2,1,6,3,5,4,5,4,...,c)51–100$,a)Facebook\n\nI would like to try out new soci...,b) $100,6,5,a)Male,28,a)Some school but no degree,a)Employed full-time,"b) $20,000 to $34,999"
4,3.0,Disadvantaged Customer,2,4,4,4,4,6,4,2,...,a)0$,a)Instagram,a) $80,4,3,a,25,a)Some school but no degree,a,"b) $20,000 to $34,999"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,95.0,Disadvantaged Customer,6,5,5,4,3,4,2,4,...,c)51–100$,a)Instagram,b) $100,5,4,a,31,a)Some school but no degree,a) Employed full-time,"c) $35,000 to $49,999"
97,96.0,Same Price,4,5,4,5,6,6,4,3,...,c)51–100$,a) Facebook,b) $100,3,5,a,23,a)Some school but no degree,a)Employed full-time,"c) $35,000 to $49,999"
98,97.0,Favored Customer,5,2,3,5,4,4,5,6,...,a)0$,a)\nb)\n\nI rarely use Instagram and Snapchat.,b) $100,5,3,a,39,a)Some school but no degree,a)Employed full-time,"f) $100,000 or more"
99,98.0,Same Price,6,6,5,3,6,4,5,4,...,c)51–100$,a)Facebook and Twitter,b) $100,6,4,b)Female,25,a)Some school but no degree,a)Employed full-time,"c) $35,000 to $49,999"


In [73]:
first_row = short_col_df.loc[[0]].rename(index={0: "groups"}).T

def column_group_index(group_name):
    return first_row.index[first_row["groups"] == group_name]

column_group_index("01 Distributive Price Fairness Perception")

Index(['001', '002', '003'], dtype='object')

In [75]:
# Remove first row
df = short_col_df.loc[1:len(short_col_df)].copy()

for col in ["{0:03}".format(i) for i in range(1, 29)]:
    df[col] = df[col].str.extract(r'^([a-zA-Z0-9]+).*', expand=False)

df

,Tri,Sce,001,002,003,004,005,006,007,008,...,019,020,021,022,023,024,025,026,027,028
1,0.0,Disadvantaged Customer,4,5,4,5,6,5,5,5,...,d,a,b,2,4,a,24,a,a,e
2,1.0,Disadvantaged Customer,2,4,5,4,3,5,5,6,...,c,a,b,5,5,a,25,d,a,d
3,2.0,Disadvantaged Customer,2,1,6,3,5,4,5,4,...,c,a,b,6,5,a,28,a,a,b
4,3.0,Disadvantaged Customer,2,4,4,4,4,6,4,2,...,a,a,a,4,3,a,25,a,a,b
5,4.0,Disadvantaged Customer,5,5,4,3,3,2,6,6,...,a,a,b,5,4,a,25,b,a,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,95.0,Disadvantaged Customer,6,5,5,4,3,4,2,4,...,c,a,b,5,4,a,31,a,a,c
97,96.0,Same Price,4,5,4,5,6,6,4,3,...,c,a,b,3,5,a,23,a,a,c
98,97.0,Favored Customer,5,2,3,5,4,4,5,6,...,a,a,b,5,3,a,39,a,a,f
99,98.0,Same Price,6,6,5,3,6,4,5,4,...,c,a,b,6,4,b,25,a,a,c


In [77]:
def mean(s):
    return sum(s)/len(s)

df["Price Fairness"] = mean([df[i].astype('int') for i in column_group_index("01 Distributive Price Fairness Perception")])
df["Purchase Intention"] = mean([df[i].astype('int') for i in column_group_index("03 Purchase Intention")])
df["Future Search Intention"] = mean([df[i].astype('int') for i in column_group_index("04 Future Search Intention")])
df["Product Involvement"] = mean([df[i].astype('int') for i in column_group_index("08 Product Involvement")])
df

,Tri,Sce,001,002,003,004,005,006,007,008,...,023,024,025,026,027,028,Price Fairness,Purchase Intention,Future Search Intention,Product Involvement
1,0.0,Disadvantaged Customer,4,5,4,5,6,5,5,5,...,4,a,24,a,a,e,4.333333,5.333333,4.333333,4.428571
2,1.0,Disadvantaged Customer,2,4,5,4,3,5,5,6,...,5,a,25,d,a,d,3.666667,4.000000,5.333333,5.000000
3,2.0,Disadvantaged Customer,2,1,6,3,5,4,5,4,...,5,a,28,a,a,b,3.000000,4.000000,4.666667,4.142857
4,3.0,Disadvantaged Customer,2,4,4,4,4,6,4,2,...,3,a,25,a,a,b,3.333333,4.666667,4.000000,4.142857
5,4.0,Disadvantaged Customer,5,5,4,3,3,2,6,6,...,4,a,25,b,a,f,4.666667,2.666667,5.000000,4.285714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,95.0,Disadvantaged Customer,6,5,5,4,3,4,2,4,...,4,a,31,a,a,c,5.333333,3.666667,3.333333,4.714286
97,96.0,Same Price,4,5,4,5,6,6,4,3,...,5,a,23,a,a,c,4.333333,5.666667,3.666667,4.428571
98,97.0,Favored Customer,5,2,3,5,4,4,5,6,...,3,a,39,a,a,f,3.333333,4.333333,4.666667,4.000000
99,98.0,Same Price,6,6,5,3,6,4,5,4,...,4,b,25,a,a,c,5.666667,4.333333,3.666667,4.000000


In [26]:
variablen = [
    "Price Fairness",
    "Purchase Intention",
    "Future Search Intention",
    "Product Involvement"
]


n = len(variablen)
# Correlation matrix
cor_matrix = pd.DataFrame(index=variablen, columns=variablen)

# Significance
p_values = pd.DataFrame(index=variablen, columns=variablen)

for i in range(n):
    for j in range(n):
        if i == j:
            cor_matrix.iloc[i, j] = "1.000"
        else:
            r, p = pearsonr(df[variablen[i]], df[variablen[j]])
            stars = "***" if p < 0.01 else "**" if p < 0.05 else "*" if p < 0.10 else ""
            cor_matrix.iloc[i, j] = f"{r:.3f}{stars}"

means = df[variablen].mean().round(2).tolist()
stds = df[variablen].std().round(2).tolist()

cor_matrix.loc["Mean"] = means
cor_matrix.loc["Standard Deviation"] = stds


In [27]:
cor_matrix

,Price Fairness,Purchase Intention,Future Search Intention,Product Involvement
Price Fairness,1.000,-0.017,0.059,0.109
Purchase Intention,-0.017,1.000,0.033,0.083
Future Search Intention,0.059,0.033,1.000,0.009
Product Involvement,0.109,0.083,0.009,1.000
Mean,3.73,4.37,4.38,4.39
Standard Deviation,1.03,0.71,0.6,0.46
